In [36]:
!pip install -U tensorflow

In [54]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# organise data

In [59]:
df = pd.read_csv('../raw_data/cassava-leaf-disease-classification/train.csv')
df.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [60]:
df.label.unique()

array([0, 3, 1, 2, 4])

In [61]:
import json
label_map = json.load(open('../raw_data/cassava-leaf-disease-classification/label_num_to_disease_map.json'))
label_map


{'0': 'Cassava Bacterial Blight (CBB)',
 '1': 'Cassava Brown Streak Disease (CBSD)',
 '2': 'Cassava Green Mottle (CGM)',
 '3': 'Cassava Mosaic Disease (CMD)',
 '4': 'Healthy'}

In [68]:
label_map = {'0': 'cassava_bacterial_blight',
 '1': 'cassava_brown_streak_disease',
 '2': 'cassava_green_mottle',
 '3': 'cassava_mosaic_disease',
 '4': 'healthy'}

In [69]:
df['label_text'] = df.label.astype(str).map(label_map)

In [72]:
# run once to reorganise images into folders of their class
# create new folders
#for label in label_map.values():
#    os.mkdir(f'../raw_data/cassava-leaf-disease-classification/train_images/{label}')

# for idx, row in df.iterrows():

#     old_path = f"../raw_data/cassava-leaf-disease-classification/train_images/{row['image_id']}"
#     new_path = f"../raw_data/cassava-leaf-disease-classification/train_images/{row['label_text']}/{row['image_id']}"
    
#     os.rename(old_path, new_path)

# Get Data

In [40]:
labels_list = df.sort_values('image_id').label.values.tolist()


In [80]:
#'/Users/alecsharp/me/cassava_farmer/raw_data/cassava-leaf-disease-classification/train_images'

train_path = '../raw_data/cassava-leaf-disease-classification/train_images'

from tensorflow.keras.utils import image_dataset_from_directory

ds = image_dataset_from_directory(
    train_path, batch_size=32
)

Found 21397 files belonging to 5 classes.
